In [90]:
# General utilities
import requests
import yaml

# geospatial
import shapely 
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, MultiPoint, LineString, Polygon, MultiPolygon
from descartes import PolygonPatch

# data transformation
import numpy as np
import pandas as pd
import networkx as nx

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Extra nifty methods
from collections import Counter

# Custom functions
#import utils.utils as ut

## Utilities

In [95]:
## ADJUSTMENT OF A PACKAGE FUNCTION
def get_edge_colors_by_attr(G, attr, num_bins=5, cmap='viridis', start=0, stop=1, na_color='none'):
    """
    Get a list of edge colors by binning some continuous-variable attribute into
    quantiles.
    Parameters
    ----------
    G : networkx multidigraph
    attr : string
        the name of the continuous-variable attribute
    num_bins : int
        how many quantiles
    cmap : string
        name of a colormap
    start : float
        where to start in the colorspace
    stop : float
        where to end in the colorspace
    na_color : string
        what color to assign nodes with null attribute values
    Returns
    -------
    list
    """
    if num_bins is None:
        num_bins=len(G.edges())
    bin_labels = range(num_bins)
    attr_values = pd.Series([data[attr] for u, v, key, data in G.edges(keys=True, data=True)])
    cats = pd.qcut(x=attr_values.rank(method='first'), q=num_bins, labels=bin_labels)
    colors = ox.get_colors(num_bins, cmap, start, stop)
    edge_colors = [colors[int(cat)] if pd.notnull(cat) else na_color for cat in cats]
    return edge_colors


## CUSTOM FUNCTIONS
def try_shortest_path(origin, destination, G):
  """
  Function that gets shortest path with Networkx if it exists. 
  Returns [0] if there is no shortest path instead of an error. 
  Uses Dkijstra's algorithm by default. None others accepted at present
  Parameters
  ----------
  origin:
  destination:
  G:
  Returns
  -------
  goodpath
  """  
  try:
    goodpath = nx.shortest_path(G, origin, destination)
    return(goodpath)
  except:
    return([0])


def path_to_edge(path, G):
    edge_nodes = list(zip(path[:-1], path[1:]))
    return(edge_nodes)

## Get Data

In [5]:
# Define bounding box (W, S, E, N) for the area of Wellington we're interested in
# Copied from http://boundingbox.klokantech.com/
general_bbox = [174.5813,-41.4552,175.0722,-41.1527]

# Separate out the bounding box list into 4 vertices. 
south = general_bbox[1]
west = general_bbox[0]
north = general_bbox[3]
east = general_bbox[2]

# Set OSM bounding box
osm_bbox = [south, west, north, east]

# Open the API keys stored in a YAML file
with open("../utils/api_keys.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)

# Get Google Elevation API key
google_elevation_api_key = data_loaded['google_elevation_api_key'][0]

In [6]:
G_drive = ox.graph_from_bbox(north, south, east, west, network_type='drive')
# Split the graph into node and edge dataframes 
G_drive_nodes_df, G_drive_edges_df = ox.graph_to_gdfs(G_drive)

In [7]:
G_walk = ox.graph_from_bbox(north, south, east, west, network_type='walk')
# Split the graph into node and edge dataframes 
G_walk_nodes_df, G_walk_edges_df = ox.graph_to_gdfs(G_walk)

## Road network connectivity

## Basic routing

## Accessibility analysis

## Flows from routes
![](../assets/wgtn_shape.png)

In [ ]:
# get the place shape
gdf = ox.gdf_from_place('Wellington City')
gdf = ox.project_gdf(gdf, to_crs='epsg:4167')

# Can also do the following to re-project Wellington shape file
# gdf['geometry'] = gdf['geometry'].to_crs(epsg=4167)

In [10]:
wgtn_stn = (-41.27803, 174.7811)
wgtn_stn_node = ox.get_nearest_node(G_drive, wgtn_stn)
test = G_drive_nodes_df.head(4).reset_index()
test['destination'] = wgtn_stn_node
x = test['osmid']
y = test['destination']
test_routes =  list(map(tryconvert, 
                        test['osmid'],
                        test['destination'], 
                        G_drive))


In [11]:
p = nx.shortest_path(G_drive, target=wgtn_stn_node)
df_p = list(p.values())

In [14]:
edge_path =  list(map(path_to_edge, df_p, G_drive)) 

test_paths = [j for i in edge_path for j in i]
count_occurrences = Counter(test_paths)

for u, v, k, data in G_drive.edges(data=True, keys=True):
  data["occurrences"] = count_occurrences[u,v]

In [15]:
H = nx.MultiDiGraph()
for u,v,attr in G_drive.edges(data=True):
    if attr['occurrences'] > 100:
        H.add_edge(u,v, occurrences=attr['occurrences'], attr_dict=attr)
        H.node[u].update(G_drive.node[u])
        H.node[v].update(G_drive.node[v])
        
H.graph['crs'] = G_drive.graph['crs']
H.graph['name'] = G_drive.graph['name']

In [108]:
G_nodes_flow_df, G_edges_flow_df = ox.graph_to_gdfs(H)
ec = get_edge_colors_by_attr(H, 'occurrences', cmap='viridis')
fig, ax = ox.plot_graph(H, fig_height=10, edge_color= ec, edge_linewidth=2, node_size=0, show=False, close=False)

# Create colourbar
cmap = mpl.cm.get_cmap('viridis')
normalize = mpl.colors.Normalize(vmin=min(G_edges_flow_df['occurrences']),vmax=max(G_edges_flow_df['occurrences']))

# Add colourbar to axis
cax, _ = mpl.colorbar.make_axes(ax, location='right')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=normalize)

# Add Wellington Polygon as a polygon patch
wgtn_geometry = gdf['geometry'][0]
patch = PolygonPatch(wgtn_geometry, ec='k', linewidth=3, alpha=0.1, zorder=-1)
ax.add_patch(patch)

# Use Wellington bounding box to set limits
ax.set_xlim(west, east);
ax.set_ylim(south, north);


In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

